In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train='../input/petfinder-pawpularity-score/train'
test='../input/petfinder-pawpularity-score/test'

In [ ]:
import cv2
img1="../input/petfinder-pawpularity-score/train/0042bc5bada6d1cf8951f8f9f0d399fa.jpg"
img2="../input/petfinder-pawpularity-score/train/0007de18844b0dbbb5e1f607da0606e0.jpg"

plt.figure(1, figsize = (15 , 10))
plt.subplot(1 , 2 , 1)
plt.imshow(cv2.imread(img1))
plt.title('Image One')
plt.xticks([]) , plt.yticks([])

plt.subplot(1 , 2 , 2)
plt.imshow(cv2.imread(img2))
plt.title('Image Two')
plt.xticks([]) , plt.yticks([])

plt.show()

In [ ]:
train=pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
test=pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
train.shape, test.shape

In [ ]:
train['file_path']=train['Id'].apply(lambda x: "../input/petfinder-pawpularity-score/train/"+x+'.jpg')
test['file_path']=test['Id'].apply(lambda x: "../input/petfinder-pawpularity-score/test/"+x+'.jpg')

In [ ]:
train=train.head(3500)

In [ ]:
train.Pawpularity.describe()

In [ ]:
test.head()

In [ ]:
import tensorflow as tf
image_size=224
def preprocess(image_url):
    image_string=tf.io.read_file(image_url)
    image=tf.image.decode_jpeg(image_string, channels=3)
    image=tf.cast(image, tf.float32)/255.0
    image=tf.image.central_crop(image, 1.0)
    image=tf.image.resize(image, (image_size, image_size))
    return image

In [ ]:
x_train=[]
for i in train['file_path']:
    x=preprocess(i)
    x_train.append(x)
X=np.array(x_train)

In [ ]:
import gc
gc.collect()

In [ ]:
test_data=[]
for i in test['file_path']:
    x=preprocess(i)
    test_data.append(x)
test=np.array(test_data)

In [ ]:
y=train['Pawpularity']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.1, random_state=21)

In [ ]:
from keras.layers import Dense, Flatten, Conv2D, Dropout, MaxPooling2D
from keras.models import Sequential

In [ ]:
model=Sequential([
    Conv2D(64, (3, 3), padding='same', activation='relu', input_shape=(224, 224, 3)),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    Conv2D(256, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Conv2D(512, (3, 3), padding='same', activation='relu'),
    Conv2D(512, (3, 3), padding='same', activation='relu'),
    Conv2D(512, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Conv2D(512, (3, 3), padding='same', activation='relu'),
    Conv2D(512, (3, 3), padding='same', activation='relu'),
    Conv2D(512, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=(2, 2)),
    
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.50),
    Dense(1, activation='relu')
])

In [ ]:
model.summary()

In [ ]:
model.compile(
    loss='mse',
    optimizer='adam',
    metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse")]
)

In [ ]:
model.fit(X_train, y_train, validation_split=0.1, epochs=25, batch_size=64)

In [ ]:
submission=pd.read_csv('../input/petfinder-pawpularity-score/sample_submission.csv')
submission['Pred']=model.predict(test)
submission

In [ ]:
from sklearn.metrics import mean_squared_error
import math

math.sqrt(mean_squared_error(submission['Pawpularity'].values, submission['Pred'].values))

In [ ]:
submission.drop('Pawpularity', axis=1, inplace=True)
submission.columns=['Id', 'Pawpularity']
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)